In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# from sklearn.cluster import KMeans # K-meansの代わりにGMMをインポート
from sklearn.mixture import GaussianMixture
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
import japanize_matplotlib  # 日本語表示ライブラリ

# --- 1. データの読み込みと前処理 ---
try:
    df = pd.read_csv('sensor_features.csv')
except FileNotFoundError:
    print("エラー: 'sensor_features.csv' が見つかりません。")
    print("先に 'calculate_features.py' を実行してください。")
    exit()

print("データの読み込みが完了しました。")
df_new = df[df["label"].isin(["aruki", "kizyo", "typing","hand"])].reset_index(drop=True)
features_df = df_new.drop(columns=['filename', 'label'])
true_labels = df_new['label']

scaler = StandardScaler()
features_scaled = scaler.fit_transform(features_df)
print("特徴量の標準化が完了しました。")

# --- 2. GMM (混合ガウスモデル) によるクラスタリングの実行 ---
n_components = 4  # GMMではクラスタ数を n_components と呼ぶのが一般的
gmm = GaussianMixture(n_components=n_components, random_state=42, n_init=10)
predicted_clusters = gmm.fit_predict(features_scaled)
df_new['cluster'] = predicted_clusters
print(f"GMMクラスタリング (n_components={n_components}) が完了しました。")


# --- 3. 結果の評価 (クロス集計) ---
crosstab = pd.crosstab(df_new['label'], df_new['cluster'])
print("\n--- クラスタリング結果のクロス集計 ---")
print(crosstab)
print("\n[見方]")
print("・各行が実際の行動ラベル、各列がGMMによって割り当てられたクラスタ番号です。")
print("・例えば、'aruki'行のデータが'cluster 0'列に集中していれば、'cluster 0'は歩行動作を表していると解釈できます。")


# --- 4. 正答率の計算 ---
print("\n--- 正答率の計算 ---")
# 1. クラスタ番号と実際のラベルを対応付けるマップを作成
cluster_to_label_map = {}
# crosstabの各列（クラスタ）に対してループ
for cluster_num in crosstab.columns:
    # そのクラスタで最も頻繁に出現するラベルを特定
    most_frequent_label = crosstab[cluster_num].idxmax()
    cluster_to_label_map[cluster_num] = most_frequent_label

print("クラスタとラベルの対応付け:")
print(cluster_to_label_map)

# 2. 予測されたクラスタ番号を、対応付けたラベル名に変換
predicted_labels = df_new['cluster'].map(cluster_to_label_map)

# 3. 実際のラベルと予測ラベルを比較して正答率を計算
accuracy = accuracy_score(true_labels, predicted_labels)
print(f"\n正答率: {accuracy:.2%}") # パーセント表示


# --- 5. 結果の可視化 (PCAによる次元削減) ---
print("\n結果を可視化しています...")
pca = PCA(n_components=2)
features_pca = pca.fit_transform(features_scaled)

df_pca = pd.DataFrame(data=features_pca, columns=['PC1', 'PC2'])
df_pca['cluster'] = df_new['cluster']
df_pca['label'] = df_new['label']

fig, axes = plt.subplots(1, 2, figsize=(18, 8))

sns.scatterplot(ax=axes[0], x='PC1', y='PC2', hue='cluster', data=df_pca, palette='viridis', s=100, alpha=0.9)
axes[0].set_title('GMMによるクラスタリング結果 (PCAで2次元に可視化)', fontsize=14)
axes[0].set_xlabel('主成分1', fontsize=12)
axes[0].set_ylabel('主成分2', fontsize=12)
axes[0].legend(title='予測クラスタ')
axes[0].grid(True)

sns.scatterplot(ax=axes[1], x='PC1', y='PC2', hue='label', data=df_pca, palette='deep', s=100, alpha=0.9)
axes[1].set_title('実際の行動ラベル (PCAで2次元に可視化)', fontsize=14)
axes[1].set_xlabel('主成分1', fontsize=12)
axes[1].set_ylabel('主成分2', fontsize=12)
axes[1].legend(title='実際の行動')
axes[1].grid(True)

plt.suptitle(f'GMMクラスタリングの結果と実際のラベルの比較 (正答率: {accuracy:.2%})', fontsize=18)
plt.tight_layout(rect=[0, 0, 1, 0.96])

# --- ここから追加 ---
# グラフをファイルとして保存
# ファイル名は 'gmm_clustering_result.png'
# dpi=300 で高解像度に、bbox_inches='tight' で余白を自動調整して見切れを防ぎます。
plt.savefig('gmm_clustering_result.png', dpi=300, bbox_inches='tight')
print("\nグラフを 'gmm_clustering_result.png' として保存しました。")
# --- ここまで追加 ---

plt.show()

print("\n分析が完了しました。")